# Cambio climatico local

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#Descargar el archivo
data = pd.read_csv("Datos_Hermosillo_editado.txt",sep="\s+",skipfooter=1,skiprows=1,engine='python',names=['Fecha', 'Precip', 'Evap', 'Tmax', 'Tmin'])
data = data.replace('Nulo','NA')
#Cambiamos el tipo de variable de las columnas a flotante
data[['Precip','Evap','Tmax','Tmin']] = data[['Precip','Evap','Tmax','Tmin']] \
.apply(pd.to_numeric, errors='coerce')
data['NFecha'] =  pd.to_datetime(data['Fecha'], format='%d/%m/%Y')
data = data.drop('Fecha', 1)
# Crear columnas con Año y Mes extraídos de la fecha 
data['Año'] = data['NFecha'].dt.year
data['Mes'] = data['NFecha'].dt.month
# Número de años distintos data['Año'].unique(),
NumA = len(data['Año'].unique())

In [5]:
df=pd.DataFrame(data)

In [6]:
df.head()

,Precip,Evap,Tmax,Tmin,NFecha,Año,Mes
0,0.0,7.3,35.0,22.0,1974-07-01,1974,7
1,0.0,10.5,34.0,22.5,1974-07-02,1974,7
2,0.0,13.6,31.5,21.0,1974-07-03,1974,7
3,0.0,10.7,41.0,28.0,1974-07-04,1974,7
4,0.0,10.9,35.0,28.0,1974-07-05,1974,7


In [7]:
#numero de datos nulos en la columna
pd.isnull(df["Tmax"]).values.ravel().sum()

150

In [8]:
#numero de valores nulos en la columna
pd.isnull(df["Tmin"]).values.ravel().sum()

149

In [9]:
df1=df

In [10]:
df1.head()

,Precip,Evap,Tmax,Tmin,NFecha,Año,Mes
0,0.0,7.3,35.0,22.0,1974-07-01,1974,7
1,0.0,10.5,34.0,22.5,1974-07-02,1974,7
2,0.0,13.6,31.5,21.0,1974-07-03,1974,7
3,0.0,10.7,41.0,28.0,1974-07-04,1974,7
4,0.0,10.9,35.0,28.0,1974-07-05,1974,7


In [11]:
pd.isnull(df1["Tmax"]).values.ravel().sum()

150

Preparamos los datos con los que vamos a trabajar

In [12]:
#rellenamos los datos nulos con el valor mas pròximo

df1["Tmax"]=df1["Tmax"].fillna(method="ffill")
df1["Tmin"]=df1["Tmin"].fillna(method="ffill")
df1["Precip"]=df1["Precip"].fillna(method="ffill")

In [13]:
#verificamos que no tengamos datos nulos en las columnas
pd.isnull(df1["Tmax"]).values.ravel().sum()
pd.isnull(df1["Tmin"]).values.ravel().sum()
pd.isnull(df1["Precip"]).values.ravel().sum()

0

In [35]:
#calculamos el primer indice, número de días con heladas por año (FD: Tmin < 0ºC)
   
# Filtramos las temperaturas menores a 0°C

data_filter = df1[df1['Tmin']<0] 

# creamos un data frame con data filter

data1 = pd.DataFrame(data_filter.groupby('Año').count())  

#creamos un data frame con dos columnas: año y número de dias con tem<0

data2 = data1.filter(['Tmin'],axis=1) 
data2['Número de heladas']=data2['Tmin']
data2 = data2.drop(['Tmin'],1)    
data2 = data2.reset_index()    #cambiamos el indice


#Graficamos los datos obtenidos

data2.plot.bar(x='Año', y='Número de heladas')
plt.title("FD: Número de días con heladas por año")
plt.ylabel("Días con heladas") 
plt.xlabel("Años",)
plt.savefig('grafica1') 

In [15]:
data2

,Año,Número de heladas
0,1975,1
1,1976,2
2,1985,2
3,1990,3
4,2011,2


In [16]:
# 2.- Número de días de verano por año (SU: Tmax > 25ºC)


# Filtramos las temperaturas mayores a 25°C

data_filter1 = df1[df1['Tmax']>25] 

# creamos un data frame con data_filter1

data3 = pd.DataFrame(data_filter1.groupby('Año').count())  

#creamos un data frame con dos columnas: año y número de dias con tem>25

data4 = data3.filter(['Tmax'],axis=1) 
data4['#Tem>25']=data4['Tmax']
data4 = data4.drop(['Tmax'],1)    
data4 = data4.reset_index()    #cambiamos el indice
data4

#Graficamos los datos obtenidos

data4.plot.bar(x='Año', y='#Tem>25')
plt.title("FD: Número de días con temperaturas mayores de 25 por año")
plt.ylabel("Días") 
plt.xlabel("Años")
plt.savefig('grafica2') 

In [18]:
# 3.- Número de noches tropicales por año (TR: Tmin > 20ºC)

# Filtramos las temperaturas mayores a 25°C

data_filter2 = df1[df1['Tmin']>20] 

# creamos un data frame con data_filter1

data5 = pd.DataFrame(data_filter2.groupby('Año').count())  

#creamos un data frame con dos columnas: año y número de dias con Tmin>20


data6 = data5.filter(['Tmin'],axis=1) 
data6['#Tem>20']=data6['Tmin']
data6 = data6.drop(['Tmin'],1)    
data6 = data6.reset_index()    #cambiamos el indice
data6

#Graficamos los datos obtenidos

data6.plot.bar(x='Año', y='#Tem>20')
plt.title("FD: Número de noches con temperaturas mayores a 20°C por año")
plt.ylabel("Noches") 
plt.xlabel("Años")
plt.savefig('grafica3') 

In [22]:
# 4.- Longitud de la estación de cultivo por año (GSL: Periodo entre
# los primeros 6 días seguidos del año  Tprom > 5ºC, y los últimos 6
# días seguidos del año con Tprom < 5ºC)

df['TEMPPROM']=(df['Tmin']+df['Tmax'])/2

Tprom = pd.DataFrame( df.filter(['TEMPPROM','Año'],axis=1) )

nT = Tprom[Tprom['TEMPPROM']<5]

Primeros = []
indice = -1

for j in range(Tprom["Año"][0],Tprom["Año"][len(df1)-1]+1):
    for i in range(5,len(df1)-1):
        if(Tprom["Año"][i-5] == j):
            if(Tprom["TEMPPROM"][i-5] > 5 and Tprom["TEMPPROM"][i-4] > 5 and Tprom["TEMPPROM"][i-3] > 5 and
               Tprom["TEMPPROM"][i-2] > 5 and Tprom["TEMPPROM"][i-1] > 5 and Tprom["TEMPPROM"][i]   > 5):
                    indice = i
                    break
    if(indice != -1):
        Primeros.append(indice)
        PrimerosSeis= -1
    else:
        Primeros.append("NA")
        
        
#Tprom > 5ºC, y los últimos 6 días seguidos del año con Tprom < 5ºC)

Ultimos = []
indice = -1

for j in range(Tprom["Año"][len(df1)-1],Tprom["Año"][0]-1,-1):
    for i in range(len(df1)-1,5,-1):
        if(Tprom["Año"][i] == j):
            if(Tprom["TEMPPROM"][i-5] < 5 and Tprom["TEMPPROM"][i-4] < 5 and Tprom["TEMPPROM"][i-3] < 5 and
               Tprom["TEMPPROM"][i-2] < 5 and Tprom["TEMPPROM"][i-1] < 5 and Tprom["TEMPPROM"][i]   < 5):
                    indice = i-5
                    break
    if(indice != -1):
        Ultimos.append(indice)
        indice = -1
    else:
        Ultimos.append("NA")
        
Ultimos = Ultimos[::-1]

#Si la temperatura nunca es mayor a los 5 grados, interpretar la temporada de cultivo como nula
GSL = []
inicial = Tprom['Año'][0]
for i in range(0,len(Ultimos)):
    if(Ultimos[i]!="NA" and Primeros[i]!="NA"):
        GSL.append(Ultimos[i]-Primeros[i])
    if(Ultimos[i]=='NA'):
            GSL.append( len(Tprom[ Tprom['Año'] == inicial].index) )
    else:
        if(Primeros[i]=='NA'):
            GSL.append(0)
    inicial=inicial+1
    
    

 


In [23]:
# 5.- TXx: La máxima mensual de la temperatura máxima, max(Tmax).

TemMax=[]
inicial=df1['Año'][0]
nAños=df1['Año'].nunique()

#El primer loop crea un DataFrame del año
for i in range(0,nAños):
        daño = df1[df1['Año']==inicial]
        
        #El segundo loop crea un DataFrame por mes
        for j in range (1,13):
                dmes = daño[daño['Mes']==j]
                
                #Se busca la máxima de Tmax 
                TmaxMes = dmes.Tmax.max()
                TemMax.append(TmaxMes)    
        inicial=inicial+1

plt.plot(TemMax)   
plt.xlabel("Meses")   
plt.ylabel("Temperatura (°C)")  
plt.title('Máximo mensual de la temperatura máxima')
plt.savefig('grafica5')


In [24]:
#6.- TNx. La máxima mensual de la temperatura mínima, max(Tmin).

TemMin=[]
inicial=df1['Año'][0]
nAños=df1['Año'].nunique()

#El primer loop crea un DataFrame del año
for i in range(0,nAños):
        daño = df1[df1['Año']==inicial]
        
        #El segundo loop crea un DataFrame por mes
        for j in range (1,13):
                dmes = daño[daño['Mes']==j]
                
                #Se busca la máxima de Tmax 
                TmaxMes = dmes.Tmin.max()
                TemMin.append(TmaxMes)    
        inicial=inicial+1

plt.plot(TemMin)   
plt.xlabel("Meses")   
plt.ylabel("Temperatura (°C)")  
plt.title('Máximo mensual de la temperatura mínima')
plt.savefig('grafica6')

In [25]:
# 7.- El mínimo mensual de la temperatura máxima, min(Tmax). 

TemMax1=[]
inicial=df1['Año'][0]
nAños=df1['Año'].nunique()

#El primer loop crea un DataFrame del año
for i in range(0,nAños):
        daño = df1[df1['Año']==inicial]
        
        #El segundo loop crea un DataFrame por mes
        for j in range (1,13):
                dmes = daño[daño['Mes']==j]
                
                #Se busca la máxima de Tmax 
                TmaxMes = dmes.Tmax.min()
                TemMax1.append(TmaxMes)    
        inicial=inicial+1

plt.plot(TemMax1)   
plt.xlabel("Meses")   
plt.ylabel("Temperatura (°C)")  
plt.title('Mínimo mensual de la temperatura máxima')
plt.savefig('grafica7')


In [26]:
# 8.- El mínimo mensual de la temperatura mínima, min(Tmin).


TemMin1=[]
inicial=df1['Año'][0]
nAños=df1['Año'].nunique()

#El primer loop crea un DataFrame del año
for i in range(0,nAños):
        daño = df1[df1['Año']==inicial]
        
        #El segundo loop crea un DataFrame por mes
        for j in range (1,13):
                dmes = daño[daño['Mes']==j]
                
                #Se busca la máxima de Tmax 
                TmaxMes = dmes.Tmin.min()
                TemMin1.append(TmaxMes)    
        inicial=inicial+1

plt.plot(TemMin1)   
plt.xlabel("Meses")   
plt.ylabel("Temperatura (°C)")  
plt.title('Mínimo mensual de la temperatura mínima')
plt.savefig('grafica8')

In [27]:
# 9.- El promedio mensual de la diferencia de temperaturas (Tmax-Tmin). 

df1['dif_T']=(df1.Tmax-df1.Tmin)#creamos una columna en el DF1 con la
# diferencia de Tmax y Tmin


DTR=[]
inicial=df1['Año'][0]
num_años=df1['Año'].nunique()

for i in range(0,num_años):
        df_año = df1[df1['Año']==inicial]
        
        for j in range (1,13):
                df_mes = df_año[df_año['Mes']==j]
                dif_T_prom = df_mes.dif_T.mean()
                DTR.append( dif_T_prom ) 
                
        inicial=inicial+1
        
        
        
plt.plot(DTR)
   
plt.xlabel("Meses")   
plt.ylabel("Temperatura (°C)")  
plt.title('Promedio mensual de la diferencia de temperaturas')
plt.savefig('grafica9')


In [28]:
# 10.- Precipitación diaria máxima mensual

PdMm = []
inicial=df1['Año'][0]
nAños=df1['Año'].nunique()

for i in range(0,nAños):
    df_año = df1[df1['Año']==inicial]
    for j in range(1,13):
        df_mes = df_año[df_año['Mes']==j]
        if(df_mes['Precip'].mean()!=0 and df_mes['Precip'].count()!=0):
            maxid = df_mes['Precip'].idxmax()
            maxim = df_mes['Precip'][maxid]
            PdMm.append(maxim)
        else:  
            PdMm.append(0.0)
            
    inicial=inicial+1
    

plt.plot(PdMm, label = "Precipitación diaria máxima mensual")   
plt.xlabel("Meses")   
plt.ylabel("Precipitación (mm)")  
#plt.legend()
plt.title('Precipitación diaria máxima mensual')
plt.savefig('grafica10')

In [29]:
# 11.- Precipitación diaria máxima mensual en 5 días consecutivos. 

PdMm5d = []
Tms = -1

primer_año=df1['Año'][0]
num_años=df1['Año'].nunique()

for i in range(0,num_años):
    df_año = df1[df1['Año']==primer_año]
    for j in range(1,13):
        df_mes = df_año[df_año['Mes']==j]
        df_mes = df_mes.reset_index()
        if(df_mes.Precip.count()!=0 and df_mes.Precip.mean()!=0):
            for k in range(4,len(df_mes)):
                lluvia = (df_mes["Precip"][k-4]+df_mes["Precip"][k-3]+
                         df_mes["Precip"][k-2]+df_mes["Precip"][k-1]+
                         df_mes["Precip"][k])
               
                if(lluvia>Tms):
                    Tms=lluvia
            
            PdMm5d.append(Tms)
            Tms=-1
        else:
            PdMm5d.append(0.0)
        
    primer_año=primer_año+1
    

plt.plot(PdMm5d)   
plt.xlabel("Meses")   
plt.ylabel("Precipitación (mm)")  
plt.title('Precipitación diaria máxima mensual en 5 días consecutivos')
plt.savefig('grafica11')

In [30]:
# 12.- Número de días en un año con precipitación mayor igual a 1mm. 

SDII=pd.DataFrame( df1.loc[df1['Precip'] >= 1 ] )
SDII=pd.DataFrame( SDII.groupby('Año').count() )
SDII_df = SDII.filter(['Precip'],axis=1)

SDII_df['dias_precip'] = SDII_df['Precip'] 
SDII_df = SDII_df.drop(['Precip'],1)             
SDII_df= SDII_df.reset_index()


SDII_df.plot.bar(x='Año',y='dias_precip') 
plt.title("Días con precipitación mayor a 1mm por año")       
plt.ylabel("Número de días")                                      
plt.xlabel("Año")                                               
plt.savefig('grafica12')

In [31]:
# 13.- Número de días en el año con precipitación diaria mayor igual a 10 mm.

R10mm=pd.DataFrame( df.loc[df['Precip'] >= 10 ] )
R10mm=pd.DataFrame( R10mm.groupby('Año').count() )


R10mm_df = R10mm.filter(['Precip'],axis=1)
R10mm_df['dias_precip_10'] = R10mm_df['Precip']
R10mm_df = R10mm_df.drop(['Precip'],1)
R10mm_df= R10mm_df.reset_index()

#13va Gráfica: R10mm
R10mm_df.plot.bar(x='Año',y='dias_precip_10')
plt.title("Días con precipitación mayor a 10mm por año")      
plt.ylabel("Número de días")                                      
plt.xlabel("Año")                                               
plt.savefig('grafica13')


In [32]:
# 14.- Número de días en el año con precipitación diaria mayor igual a 20mm. 

R20mm=pd.DataFrame( df.loc[df['Precip'] >= 20 ] )
R20mm=pd.DataFrame( R20mm.groupby('Año').count() )

R20mm_df = R20mm.filter(['Precip'],axis=1)
R20mm_df['dias_precip_20'] = R20mm_df['Precip']
R20mm_df = R20mm_df.drop(['Precip'],1)
R20mm_df= R20mm_df.reset_index()

#14va Gráfica: R20mm
R20mm_df.plot.bar(x='Año',y='dias_precip_20')
plt.title("Días con precipitación mayor a 20mm por año")      
plt.ylabel("Número de días")                                      
plt.xlabel("Año")                                               
plt.savefig('grafica14')

In [33]:
# 15.- Número de días consecutivos secos, sin precipitación
# o precipitación < 1mm.
'''
CDD_año = []
CDD = []
dias = 0

inicial=df1['Año'][0]
nAños=df1['Año'].nunique()

for i in range(0,nAños):
    daño = df[df['Año']==inicial]
    daño = daño.reset_index()
    for k in range(0,len(daño)):
        if(daño['Precip'][k]<1):
            dias = dias +1
        else:
            CDD_año.append(dias)
            dias=0
    CDD_max=max(CDD_año)
    CDD_año = []
    CDD.append(CDD_max)
    inicial=inicial+1
     


AÑOS = [inicial+i for i in range(0, nAños)]


X = AÑOS
N = np.arange(len(X))
Y = CDD
 
plt.bar(N, Y,  width=0.5,align='center', alpha=0.7,color='#D2691E')
plt.xticks(N, X, size = 'small', color = 'k', rotation = 90)
plt.title('Número de días consecutivos secos.')
plt.xlabel('Años')
plt.ylabel('Número de días')
plt.savefig('grafica15')

'''

"\nCDD_año = []\nCDD = []\ndias = 0\n\ninicial=df1['Año'][0]\nnAños=df1['Año'].nunique()\n\nfor i in range(0,nAños):\n    daño = df[df['Año']==inicial]\n    daño = daño.reset_index()\n    for k in range(0,len(daño)):\n        if(daño['Precip'][k]<1):\n            dias = dias +1\n        else:\n            CDD_año.append(dias)\n            dias=0\n    CDD_max=max(CDD_año)\n    CDD_año = []\n    CDD.append(CDD_max)\n    inicial=inicial+1\n     \n\n\nAÑOS = [inicial+i for i in range(0, nAños)]\n\n\nX = AÑOS\nN = np.arange(len(X))\nY = CDD\n \nplt.bar(N, Y,  width=0.5,align='center', alpha=0.7,color='#D2691E')\nplt.xticks(N, X, size = 'small', color = 'k', rotation = 90)\nplt.title('Número de días consecutivos secos.')\nplt.xlabel('Años')\nplt.ylabel('Número de días')\nplt.savefig('grafica15')\n\n"

In [34]:
#16.- Número de días consecutivos húmedos, con precipitación igual o mayor a 1mm. 

'''
CWD_año = []
CWD = []
dias = 0

inicial=df1['Año'][0]
nAños=df1['Año'].nunique()

for i in range(0,nAños):
    daño = df1[df1['Año']==inicial]
    daño = daño.reset_index()
    for k in range(0,len(daño)):
        if(daño['Precip'][k]>=1):
            dias = dias +1
        else:
            CWD_año.append(dias)
            dias=0
    CWD_max=max(CWD_año)
    CWD_año = []
    CWD.append(CWD_max)
    inicial=inicial+1


AÑOS = [inicial+i for i in range(0, nAños)]

print(AÑOS)
len(AÑOS)
print(CWD)
len(CWD)


X = AÑOS
N = np.arange(len(X))
Y = CWD
 
plt.bar(N, Y,  width=0.5,align='center', alpha=0.7,color='#778899')
plt.xticks(N, X, size = 'small', color = 'k', rotation = 90)
plt.title('CWD: Número de días consecutivos húmedos.')
plt.xlabel('Años')
plt.ylabel('Número de días')
plt.savefig('grafica16')
'''


"\nCWD_año = []\nCWD = []\ndias = 0\n\ninicial=df1['Año'][0]\nnAños=df1['Año'].nunique()\n\nfor i in range(0,nAños):\n    daño = df1[df1['Año']==inicial]\n    daño = daño.reset_index()\n    for k in range(0,len(daño)):\n        if(daño['Precip'][k]>=1):\n            dias = dias +1\n        else:\n            CWD_año.append(dias)\n            dias=0\n    CWD_max=max(CWD_año)\n    CWD_año = []\n    CWD.append(CWD_max)\n    inicial=inicial+1\n\n\nAÑOS = [inicial+i for i in range(0, nAños)]\n\nprint(AÑOS)\nlen(AÑOS)\nprint(CWD)\nlen(CWD)\n\n\nX = AÑOS\nN = np.arange(len(X))\nY = CWD\n \nplt.bar(N, Y,  width=0.5,align='center', alpha=0.7,color='#778899')\nplt.xticks(N, X, size = 'small', color = 'k', rotation = 90)\nplt.title('CWD: Número de días consecutivos húmedos.')\nplt.xlabel('Años')\nplt.ylabel('Número de días')\nplt.savefig('grafica16')\n"